In [ ]:

ppmsdict={}

for cl in ["GM12878", "HEPG2", "IMR90", "H1ESC"]:
    tfile="/mnt/lab_data2/anusri/chrombpnet/results/tobias/ATAC_PE/GM12878/GM12878_ATAC_TOBIAS_NEW_COUNTS/counts.ppms.txt"
    ppms=open(tfile.replace("GM12878", cl)).readlines()
    if cl not in ppmsdict:
        ppmsdict[cl] = {}
    for i in range(0,len(ppms),5):
        #print(ppms[i])
        #print()
        if ppms[i] not in ppmsdict[cl]:
            ppmsdict[cl][ppms[i]] = ppms[i+1:i+5]

    print(len(ppmsdict[cl]))

In [47]:
import pandas as pd
import os
import multiprocessing

celline = ["GM12878", "K562", "HEPG2", "H1ESC"]


tsvf = "/oak/stanford/groups/akundaje/projects/chromatin-atlas-2022/chrombpnet/script/"

annotation = pd.read_csv("atac_to_chip_1.txt",sep="\t", header=None)
anng = annotation.groupby(by=[0])

ppms="/mnt/lab_data2/anusri/chrombpnet/results/tobias/ATAC_PE/GM12878/GM12878_ATAC_TOBIAS_NEW_COUNTS/counts.ppms.txt"

fields=[]
for cellty in celline:
	cl=cellty

	count = 0
	dirs="/mnt/lab_data2/anusri/chrombpnet/results/tobias/ATAC_PE/"+cl+"/"+cl+"_ATAC_TOBIAS_NEW_COUNTS/"
	folders =os.listdir(dirs)
	dictss = {}
	for filed in folders:
		if filed.startswith("neg"):
			continue
		vals = filed.split(".")
		dictss[vals[1]] = filed

	annt = anng.get_group(cellty).reset_index()
	#print(annt.head())
	ttf = tsvf+cellty.lower()+"_chipseq.tsv"
	ppf = pd.read_csv(ttf, sep=',', header=None)

	for i,r in annt.iterrows():
		#print(r)
		vals = list(set(ppf[ppf[1].isin(r[1].split("."))][0]))
		if len(vals)==0 and r[1]=="None":
			continue
		else:					
			count+=1
			for encid in vals:

				arguments_list = []
				motif = dictss["pattern_"+str(i)]
				outf = "new_output_jan_18/"+cl+"/"+"pattern_"+str(i)+"_"+r[1]+"/"+encid+".bed"
				#if not os.path.isfile(outf):
				patternid="pos_"+motif.split("_")[1]+"_"+motif.split("_")[2].replace(".mouse","")
				#+"_"+motif.split("_")[3]
				print(cl,"pattern_"+patternid.split(".")[-1], encid)
				fields.append([cl,"pattern_"+patternid.split(".")[-1], encid])
				#print(patternid)
				#print(cl, motif, outf, encid)

GM12878 pattern_IRF1 ENCSR000BGY
GM12878 pattern_CTCF ENCSR000AKB
GM12878 pattern_CTCF ENCSR000DKV
GM12878 pattern_CTCF ENCSR000DZN
GM12878 pattern_CTCF ENCSR000DRZ
GM12878 pattern_SPI1 ENCSR000BGQ
GM12878 pattern_RUNX1 ENCSR000BRI
GM12878 pattern_JDP2 ENCSR000BGT
GM12878 pattern_JDP2 ENCSR897MMC
GM12878 pattern_NFKB1 ENCSR387QUV
GM12878 pattern_IRF4 ENCSR000BGQ
GM12878 pattern_IRF4 ENCSR000BGY
GM12878 pattern_Gabpa ENCSR331HPA
GM12878 pattern_KLF12 ENCSR000BHK
GM12878 pattern_NFYB ENCSR000DNN
GM12878 pattern_REL ENCSR387QUV
GM12878 pattern_BATF ENCSR000BGT
GM12878 pattern_NRF1 ENCSR000DZO
GM12878 pattern_POU5F1 ENCSR000BGP
GM12878 pattern_FOSL1JUND ENCSR000BJY
GM12878 pattern_FOSL1JUND ENCSR000BUF
GM12878 pattern_NFKB2 ENCSR387QUV
GM12878 pattern_BATF ENCSR000BGT
GM12878 pattern_ZNF76 ENCSR936XTK
GM12878 pattern_ZNF76 ENCSR000DZL
GM12878 pattern_COE1 ENCSR000DZQ
GM12878 pattern_COE1 ENCSR000BGU
GM12878 pattern_PAX2 ENCSR000BHJ
GM12878 pattern_PAX2 ENCSR000BHD
K562 pattern_GATA3 ENCSR0

In [50]:
df = pd.DataFrame(fields)
df

,0,1,2
0,GM12878,pattern_IRF1,ENCSR000BGY
1,GM12878,pattern_CTCF,ENCSR000AKB
2,GM12878,pattern_CTCF,ENCSR000DKV
3,GM12878,pattern_CTCF,ENCSR000DZN
4,GM12878,pattern_CTCF,ENCSR000DRZ
...,...,...,...
133,H1ESC,pattern_CTCF,ENCSR000AMF
134,H1ESC,pattern_CTCF,ENCSR000BNH
135,H1ESC,pattern_CTCF,ENCSR000DLK
136,H1ESC,pattern_Gabpa,ENCSR000BIW


In [51]:
df.to_csv("/oak/stanford/groups/akundaje/anusri/chrombpnet_zenodo_uploads_reformatted/ChromBPNet_TF_Chip_Compare/motif_to_chip_mapping.tsv", sep="\t")

